In [71]:
import os
import pandas as pd
import numpy as np
import json

In [72]:
# Get the current working directory
CURRENT_DIRECTORY = os.getcwd()
# Get the parent directory
PARENT_DIRECTORY = os.path.dirname(CURRENT_DIRECTORY)
PARENT_DIRECTORY = os.path.dirname(PARENT_DIRECTORY)

# Open the config file and load its content into a dictionary
config_file = open(PARENT_DIRECTORY + '\\config\\config.json')
CONFIG_DATA = json.load(config_file)

# Close the file after loading the data
config_file.close()

In [73]:
WINDOW_SIZE = CONFIG_DATA['WINDOW_SIZE']
NUMBER_OF_FEATURES = 8

PITCH_THRESHOLD = CONFIG_DATA['SML_STEPS_PITCH_ROTATION_THRESHOLD']

CLASSIFICATIONS = CONFIG_DATA['CLASSES_MOTIONSPEED']

LABEL_COLUMN = 'Class_MotionSpeed'

# processed data folder path
DATA_FOLDER = PARENT_DIRECTORY + '\\processed-training-data\\4-PROCESSED-DATA\TRAIN2\\'

# save numpy data folder
NUMPY_DATA_FOLDER_FILE_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\'

# v0 data (not normalized, use to generate more data)
ALL_X_TRAIN_CLASSIFY_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\training\\ALL-X-TRAIN-CLASSIFY-MOTIONSPEED-SML-STEPS-V0.npy'
ALL_Y_TRAIN_CLASSIFY_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\training\\ALL-Y-TRAIN-CLASSIFY-MOTIONSPEED-SML-STEPS-V0.npy'

In [74]:
LABEL_TO_CATEGORY = {label: category for category, label in enumerate(CLASSIFICATIONS)}

print(LABEL_TO_CATEGORY)

{'SLOW': 0, 'AVERAGE': 1, 'FAST': 2}


In [75]:
file_names = [file for file in os.listdir(DATA_FOLDER) if file.endswith('.xlsx') and "STEPS-LR-SML" in file in file and os.path.isfile(os.path.join(DATA_FOLDER, file))]

print(file_names)

['PROC-TRAIN2-STEPS-LR-SML-110BPM.xlsx', 'PROC-TRAIN2-STEPS-LR-SML-120BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-SML-160BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-SML-220BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-SML-30BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-SML-40BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-SML-55BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-SML-60BPM.xlsx', 'PROC-TRAIN2-STEPS-LR-SML-80BPM.xlsx']


In [76]:
def normalizeSensorData(sensor_input):
    return (sensor_input + 180)/360

In [77]:
def count_sequences_above_threshold(data, threshold):
    num_sensors = data.shape[1]
    counts = np.zeros(num_sensors, dtype=int)

    for sensor_index in range(num_sensors):
        count = 0
        sequence_above_threshold = False

        for value in data[:, sensor_index]:
            if value > threshold:
                if not sequence_above_threshold:
                    count += 1
                    sequence_above_threshold = True
            else:
                sequence_above_threshold = False

        counts[sensor_index] = count

    return counts

In [78]:
def df2Xy(df, windowSize=5):
  X = []
  y = []

  # NORMALIZE THE SENSOR DATA
  #df["L_Pitch_Delta"] = df["L_Pitch_Delta"].apply(normalizeSensorData)
  #df["L_Roll_Delta"] = df["L_Roll_Delta"].apply(normalizeSensorData)
  #df["R_Pitch_Delta"] = df["R_Pitch_Delta"].apply(normalizeSensorData)
  #df["R_Roll_Delta"] = df["R_Roll_Delta"].apply(normalizeSensorData)

  for i in range( len(df) - windowSize + 1):
    # inputs: X rows
    # form a new input which has size of our windowSize
    input_data_list_deltas = []
    input_data_list_readings = []

    # loop through each row in our windowsize
    for j in range(windowSize):
        # fetch sensor data for this row
        row_values_deltas = df.loc[i + j, ['L_Pitch_Delta', 'L_Roll_Delta', 'R_Pitch_Delta', 'R_Roll_Delta']].values.tolist()
        row_values_readings = df.loc[i + j, ['L_Pitch', 'L_Roll', 'R_Pitch', 'R_Roll']].values.tolist()

        # add row values to the input
        input_data_list_deltas.append(row_values_deltas)
        input_data_list_readings.append(row_values_readings)

    # turnn list into array to do arthimetic
    raw_sensor_data = np.array(input_data_list_readings)
    delta_sensor_data = np.array(input_data_list_deltas)

    # remove all negative sinces we want to the total change (we dont care which direction)
    input_calculate_deltas = np.abs(delta_sensor_data)

    # calculate the total change for each sennsor value (TOTAL POSITIVE SENSOR CHANGES)
    input_calculate_deltas = np.sum(input_calculate_deltas, axis=0)

    # calculate total sequences
    input_total_sequences = count_sequences_above_threshold(raw_sensor_data, PITCH_THRESHOLD)

    input_data_array = np.concatenate((input_calculate_deltas, input_total_sequences))

    if(np.isnan(input_data_array).any() == False):
      # turn back to list
      input_data_list = input_data_array.tolist()

      # add our input to our total inputs, marked as X
      X.append(input_data_list)

      # outputs: y labels
      label = df.loc[i + (windowSize - 1), [LABEL_COLUMN]].values.tolist()

      y.append(label)

  return (np.array(X), np.array(y))

In [79]:
ALL_X_TRAIN = np.empty((0, NUMBER_OF_FEATURES))  # List to store all X training data
ALL_Y_TRAIN = np.empty((0))  # List to store all Y training data

In [80]:
def processData(ALL_X, ALL_Y):
    for fileName in file_names:
        # Read the Excel file
        df = pd.read_excel(DATA_FOLDER + fileName)

        xTrain, yTrain = df2Xy(df, WINDOW_SIZE)

        yTrain = yTrain.reshape(-1)
        
        ALL_X = np.concatenate([ALL_X, xTrain])
        ALL_Y = np.concatenate([ALL_Y, yTrain])
        
        print('xTrain.shape:', xTrain.shape)
        print('ALL_X_TRAIN.shape:', ALL_X.shape, ' ALL_Y_TRAIN.shape:', ALL_Y.shape)
        print('-------------------------------')

    # Create the folder if it doesn't exist
    if not os.path.exists(NUMPY_DATA_FOLDER_FILE_PATH):
        os.makedirs(NUMPY_DATA_FOLDER_FILE_PATH)

    # convert to numerical labels (originally text labels)
    numerical_label = np.vectorize(LABEL_TO_CATEGORY.get)(ALL_Y)

    # Assuming your numpy array is called 'data_array'
    np.save(ALL_X_TRAIN_CLASSIFY_PATH, ALL_X)
    np.save(ALL_Y_TRAIN_CLASSIFY_PATH, numerical_label)

In [81]:
processData(ALL_X_TRAIN, ALL_Y_TRAIN)

ALL_X_TRAIN_CLASSIFY = np.load(ALL_X_TRAIN_CLASSIFY_PATH)
ALL_Y_TRAIN_CLASSIFY = np.load(ALL_Y_TRAIN_CLASSIFY_PATH)

print(ALL_X_TRAIN_CLASSIFY.shape)
print(ALL_Y_TRAIN_CLASSIFY.shape)

xTrain.shape: (3997, 8)
ALL_X_TRAIN.shape: (3997, 8)  ALL_Y_TRAIN.shape: (3997,)
-------------------------------
xTrain.shape: (3285, 8)
ALL_X_TRAIN.shape: (7282, 8)  ALL_Y_TRAIN.shape: (7282,)
-------------------------------
xTrain.shape: (4350, 8)
ALL_X_TRAIN.shape: (11632, 8)  ALL_Y_TRAIN.shape: (11632,)
-------------------------------
xTrain.shape: (3996, 8)
ALL_X_TRAIN.shape: (15628, 8)  ALL_Y_TRAIN.shape: (15628,)
-------------------------------
xTrain.shape: (6590, 8)
ALL_X_TRAIN.shape: (22218, 8)  ALL_Y_TRAIN.shape: (22218,)
-------------------------------
xTrain.shape: (8720, 8)
ALL_X_TRAIN.shape: (30938, 8)  ALL_Y_TRAIN.shape: (30938,)
-------------------------------
xTrain.shape: (8012, 8)
ALL_X_TRAIN.shape: (38950, 8)  ALL_Y_TRAIN.shape: (38950,)
-------------------------------
xTrain.shape: (3286, 8)
ALL_X_TRAIN.shape: (42236, 8)  ALL_Y_TRAIN.shape: (42236,)
-------------------------------
xTrain.shape: (4351, 8)
ALL_X_TRAIN.shape: (46587, 8)  ALL_Y_TRAIN.shape: (46587,)
-

In [82]:
print(np.max(ALL_X_TRAIN_CLASSIFY))

103.0
